# This Note book is Created by Abdelrahman Fakhry Hussein and There are some Techniques and Markdowns  has been taken from prof:Andrew NG 

# 

# Import Libraries

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import tensorflow as tf
from tensorflow import keras
import keras
import zipfile
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
import tabulate
import pandas as pd
from keras.callbacks import ReduceLROnPlateau
%matplotlib inline


# Movie ratings dataset <img align="left" src="./images/film_rating.png" style=" width:40px;" >
## The data set is derived from the [MovieLens ml-latest-small](https://grouplens.org/datasets/movielens/latest/) dataset.   
[F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>]

## The dataset has Approximately 9000 movies rated by 600 users.

# Un Ziping File of Data 

In [36]:
trainpath = r'C:\Users\abdel\Downloads\ml-latest-small.zip'
files_path = r'C:\Users\abdel\Downloads'
with zipfile.ZipFile(trainpath, 'r') as zipp:
    zipp.extractall(files_path)
    

# Get Name Of Data Files in Folder Un Zipped

In [37]:
trainpath = 'C:/Users/abdel/Downloads/ml-latest-small'

for file in os.listdir(trainpath):
    print('files in path ',file)
    
    

files in path  links.csv
files in path  movies.csv
files in path  ratings.csv
files in path  README.txt
files in path  tags.csv


# Collaborative filtering learning algorithm <img align="left" src="./images/film_filter.png"     style=" width:40px;  " >

Now, you will begin implementing the collaborative filtering learning
algorithm. You will start by implementing the objective function. 

The collaborative filtering algorithm in the setting of movie
recommendations considers a set of $n$-dimensional parameter vectors
$\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)}$, $\mathbf{w}^{(0)},...,\mathbf{w}^{(n_u-1)}$ and $b^{(0)},...,b^{(n_u-1)}$, where the
model predicts the rating for movie $i$ by user $j$ as
$y^{(i,j)} = \mathbf{w}^{(j)}\cdot \mathbf{x}^{(i)} + b^{(i)}$ . Given a dataset that consists of
a set of ratings produced by some users on some movies, you wish to
learn the parameter vectors $\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},
\mathbf{w}^{(0)},...,\mathbf{w}^{(n_u-1)}$  and $b^{(0)},...,b^{(n_u-1)}$ that produce the best fit (minimizes
the squared error).

You will complete the code in cofiCostFunc to compute the cost
function for collaborative filtering. 

# Collaborative filtering cost function

The collaborative filtering cost function is given by
$$J({\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},\mathbf{w}^{(0)},b^{(0)},...,\mathbf{w}^{(n_u-1)},b^{(n_u-1)}})= \frac{1}{2}\sum_{(i,j):r(i,j)=1}(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2
+\underbrace{
\frac{\lambda}{2}
\sum_{j=0}^{n_u-1}\sum_{k=0}^{n-1}(\mathbf{w}^{(j)}_k)^2
+ \frac{\lambda}{2}\sum_{i=0}^{n_m-1}\sum_{k=0}^{n-1}(\mathbf{x}_k^{(i)})^2
}_{regularization}
\tag{1}$$
The first summation in (1) is "for all $i$, $j$ where $r(i,j)$ equals $1$" and could be written:

$$
= \frac{1}{2}\sum_{j=0}^{n_u-1} \sum_{i=0}^{n_m-1}r(i,j)*(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2
+\text{regularization}
$$

You should now write cofiCostFunc (collaborative filtering cost function) to return this cost.



# **Vectorized Implementation**

It is important to create a vectorized implementation to compute $J$, since it will later be called many times during optimization. The linear algebra utilized is not the focus of this series, so the implementation is provided. If you are an expert in linear algebra, feel free to create your version without referencing the code below. 

Run the code below and verify that it produces the same results as the non-vectorized version.

# 

In [38]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2)+ (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

# Load Data 

In [39]:
def Load_Data():
    movies = pd.read_csv('C:/Users/abdel/Downloads/ml-latest-small/movies.csv')
    movies = movies.sort_index()
    links = pd.read_csv('C:/Users/abdel/Downloads/ml-latest-small/links.csv')
    links = links.sort_index()
    ratings = pd.read_csv('C:/Users/abdel/Downloads/ml-latest-small/ratings.csv')
    tags = pd.read_csv('C:/Users/abdel/Downloads/ml-latest-small/tags.csv')
    tags = tags.sort_index()
    return movies , links,ratings,tags

In [40]:
movies , links,ratings,tags = Load_Data()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


# Data Preprocessing 

### Merge movies and links Data to Get a one DataFrame to can doing operation on it 

### Split title to Movie_title and Movie_Year to Can doing Operation and Recommendation Base on alot of Columns including Year of Movie


# 

In [41]:
def Preprocessing(movies,links):
    X=movies.merge(links,on='movieId')
    X['imdbId']=X['imdbId'].astype(float)
    X['tmdbId']=X['tmdbId'].astype(float)
    tit=[]
    year=[]
    for i in X.index:
        if str(((X['title'][i].split('(')[-1].replace(')','')).replace(" ", "")).split('–')[-1]).isnumeric()== True:    
            tit.append(','.join(X['title'][i].split('(')[:-1]))
            year.append(((X['title'][i].split('(')[-1].replace(')','')).replace(" ", "")).split('–')[-1])
        else:
            tit.append(','.join(X['title'][i].split('(')[:]))
            year.append(0)
    tit=np.array(tit)    
    year=np.array(year)    
    #X=X.drop(['title'],axis=1)
    X['title']=tit
    X['year']=year
    X['year']=X['year'].astype(int)
    return X



### merge X with ratings dependent on movieId Column to merge X and y columns to Get a specific row and columns which is in {ratings,movies,links} Data Frames to make an adjust the values of the X_training and Y_training so that there is no film that has no value in the Rating or vice versa, to ensure that there are no wrong calculations or data 
### Get Column of mean_rating to be one of the Bases in Recommendation system by Making Grouping of title 
### Also Get Column of genres to be one of the Bases in Recommendation system by Making Grouping of title to get number of X and Y row are equal 



## Split genres Column To get column of List of Movie genres like Adventure|Animation|Children|Comedy|Fantasy => ['Adventure','Animation','Children','Comedy','Fantasy'] and 
# Has Option  to 
## 1 -> append Tag column in Tags DataFrame or Not


# 

In [42]:
def Preprocessing2(X,ratings,tags,d):
    X=X.merge(ratings,on='movieId')
    
    d1=X[['title','rating']].groupby(['title'],as_index=False).count()
    d1.columns=['title','num_rating']

    d2= X[['title','rating']].groupby(['title'],as_index=False).mean()
    d2.columns=['title','mean_rating']

    
    d3=X[['title' ,'genres']].groupby(['title'],as_index=False).first()
    d3.columns=['title','genres']

    X_feat=d1.merge(d2,on='title')
    X_feat=X_feat.merge(d3,on='title')
    
    d=d[d['movieId'].isin(ratings['movieId'])]
    X_feat=X_feat.merge(d[['title','year','movieId']],on='title')
    
    m=X_feat.merge(tags[['tag','movieId']],on='movieId',how='outer')
    d4=m[['movieId' ,'tag']].groupby(['movieId'],as_index=False).first()
    X_feat=X_feat.merge(d4,on='movieId')
    
    genress=[]
    for  i in range(X_feat.shape[0]):
        if X_feat['tag'].iloc[i] == None:
            genress.append(np.array(X_feat['genres'][i].split('|')))
        else:
            genress.append(np.concatenate((np.array(X_feat['genres'][i].split('|')), np.array(X_feat['tag'].iloc[i].split(',')))))
        X_feat['genres'].iloc[i]=X_feat['genres'].iloc[i].split('|')    
    X_feat['genress']   = genress  
    X_feat.drop('tag',axis=1,inplace=True)
    return X_feat,X

In [43]:
movies,links,ratings,tags = Load_Data()
X=Preprocessing(movies,links)
d=X
X_feat,Y_ = Preprocessing2(X,ratings,tags,d)
X_featt=X_feat.copy(deep=True)

X_feat.drop(['genress'],axis=1,inplace=True)


X_featt.sort_values(by='movieId',inplace=True)
X_feat.set_index('movieId',inplace=True,drop=True)
X_feat.sort_index(inplace=True)
X_feat.drop('title',axis=1,inplace=True)


Y_.drop(['genres','imdbId','tmdbId','title','year','timestamp'],axis=1,inplace=True)
ratings=Y_.groupby(['movieId', 'userId'])['rating'].first().unstack()
ratings.sort_index(inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [44]:
mean = np.nanmean(ratings, axis=1)

In [45]:
mean.shape

(9724,)

# Doing One Hot Encoding to genres Column 

for example  if 

                 a =[['apple','banana']
                   ,['orange','apple']]
                   
One Hot Encoding=>              
                              
                                       apple  banana  orange

                                        1       1        0
                                        
                                        1       0        1


In [46]:
mlb = MultiLabelBinarizer(sparse_output=True)

X = X_feat.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(X_feat.pop('genres')),
            
                index=X_feat.index,
                columns=mlb.classes_))


if X.isnull().values.sum() > 0:
    X=X.fillna(X['mean_tmdbId'].mean())


# Here you have an option to delete year of movie from X training Data may it not help in X Data and it lead to miss leading
# OR 
# You can drop Number of rating column

In [47]:
X.drop('year',axis=1,inplace=True)
X.drop('num_rating',axis=1,inplace=True)

In [48]:
X

,mean_rating,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,3.920930,0,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3.431818,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.259615,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2.357143,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
5,3.071429,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,4.000000,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
193583,3.500000,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
193585,3.500000,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [49]:
ratings

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# $n_m  => Number of Movies in Y it is Equal Number of Row in Y.



#  $n_u  => Number of Users  in Y it is Equal Number of Columns in Y
## Get 𝑅(𝑖,𝑗)= 0  if Y(𝑖,𝑗)= Null  else  𝑅(𝑖,𝑗)= 1  
## we fill Null Value for movie i and user j with zero Rating

### The matrix $Y$ (a  $n_m \times n_u$ matrix) stores the ratings $y^{(i,j)}$. The matrix $R$ is an binary-valued indicator matrix, where $R(i,j) = 1$ if user $j$ gave a rating to movie $i$, and $R(i,j)=0$ otherwise. 
# Throughout this part of the exercise, you will also be working with the
matrices, $\mathbf{X}$, $\mathbf{W}$ and $\mathbf{b}$: 

$$\mathbf{X} = 
\begin{bmatrix}
--- (\mathbf{x}^{(0)})^T --- \\
--- (\mathbf{x}^{(1)})^T --- \\
\vdots \\
--- (\mathbf{x}^{(n_m-1)})^T --- \\
\end{bmatrix} , \quad
\mathbf{W} = 
\begin{bmatrix}
--- (\mathbf{w}^{(0)})^T --- \\
--- (\mathbf{w}^{(1)})^T --- \\
\vdots \\
--- (\mathbf{w}^{(n_u-1)})^T --- \\
\end{bmatrix},\quad
\mathbf{ b} = 
\begin{bmatrix}
 b^{(0)}  \\
 b^{(1)} \\
\vdots \\
b^{(n_u-1)} \\
\end{bmatrix}\quad
$$ 

The $i$-th row of $\mathbf{X}$ corresponds to the
feature vector $x^{(i)}$ for the $i$-th movie, and the $j$-th row of
$\mathbf{W}$ corresponds to one parameter vector $\mathbf{w}^{(j)}$, for the
$j$-th user. Both $x^{(i)}$ and $\mathbf{w}^{(j)}$ are $n$-dimensional
vectors. For the purposes of this exercise, you will use $n=10$, and
therefore, $\mathbf{x}^{(i)}$ and $\mathbf{w}^{(j)}$ have 10 elements.
Correspondingly, $\mathbf{X}$ is a
$n_m \times 10$ matrix and $\mathbf{W}$ is a $n_u \times 10$ matrix.

We will start by loading the movie ratings dataset to understand the structure of the data.
We will load $Y$ and $R$ with the movie dataset.  
We'll also load $\mathbf{X}$, $\mathbf{W}$, and $\mathbf{b}$ with pre-computed values. These values will be learned later in the lab, but we'll use pre-computed values to develop the cost model.

In [50]:

R=ratings.isnull().values
R=~np.array(R)
R = np.multiply(R, 1)


    
    
mean = np.nanmean(ratings, axis=1)
Ynorm = (ratings.T-mean).T    
print(X.shape)
W=np.random.randn(Ynorm.shape[1]   ,X.shape[1]   )
b=np.random.randn(1,Ynorm.shape[1]    )

Ynorm=pd.DataFrame(Ynorm)
Ynorm=Ynorm.fillna(0)


Ynorm=np.array(Ynorm)
X=np.array(X)
W=np.array(W)
b=np.array(b)
R=np.array(R)

X=tf.Variable(X,dtype=tf.float64,name='X')
W=tf.Variable(W,dtype=tf.float64,name='W')
b=tf.Variable(b,dtype=tf.float64,name='b')

J = cofi_cost_func_v(X, W, b, Ynorm, R, 1.5);
print(f"Cost: {J:0.2f}")
J = cofi_cost_func_v(X, W, b, Ynorm, R, 0);
print(f"Cost: {J:0.2f}")




(9724, 21)
Cost: 1004629.16
Cost: 895376.96


 Let's now train the collaborative filtering model. This will learn the parameters $\mathbf{X}$, $\mathbf{W}$, and $\mathbf{b}$. 

 The operations involved in learning $w$, $b$, and $x$ simultaneously do not fall into the typical 'layers' offered in the TensorFlow neural network package.  Consequently, the flow used in Course 2: Model, Compile(), Fit(), Predict(), are not directly applicable. Instead, we can use a custom training loop.

Recall from earlier labs the steps of gradient descent.
- repeat until convergence:
    - compute forward pass
    - compute the derivatives of the loss relative to parameters
    - update the parameters using the learning rate and the computed derivatives 
    
TensorFlow has the marvelous capability of calculating the derivatives for you. This is shown below. Within the `tf.GradientTape()` section, operations on Tensorflow Variables are tracked. When `tape.gradient()` is later called, it will return the gradient of the loss relative to the tracked variables. The gradients can then be applied to the parameters using an optimizer. 
This is a very brief introduction to a useful feature of TensorFlow and other machine learning frameworks. Further information can be found by investigating "custom training loops" within the framework of interest.
    


# 

# Here we Use PiecewiseConstantDecay To Reduce learning Rate with a specific schedule

In [51]:
iterations = 5000
lambda_ = 1

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=9e-1,decay_steps=1000,decay_rate=0.12)

optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )
    
    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )
    
    # Log periodically.
    if iter % 100 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 968211.8
Training loss at iteration 100: 12942.8
Training loss at iteration 200: 10390.8
Training loss at iteration 300: 9892.1
Training loss at iteration 400: 9691.4
Training loss at iteration 500: 9579.0
Training loss at iteration 600: 9507.3
Training loss at iteration 700: 9458.3
Training loss at iteration 800: 9423.3
Training loss at iteration 900: 9396.9
Training loss at iteration 1000: 9376.5
Training loss at iteration 1100: 9360.4
Training loss at iteration 1200: 9347.8
Training loss at iteration 1300: 9337.8
Training loss at iteration 1400: 9329.7
Training loss at iteration 1500: 9323.2
Training loss at iteration 1600: 9317.9
Training loss at iteration 1700: 9313.5
Training loss at iteration 1800: 9309.9
Training loss at iteration 1900: 9306.9
Training loss at iteration 2000: 9304.3
Training loss at iteration 2100: 9302.2
Training loss at iteration 2200: 9300.4
Training loss at iteration 2300: 9298.8
Training loss at iteration 2400: 9297.5
Training

#  Learning movie recommendations 
------------------------------

After you have finished implementing the collaborative filtering cost
function, you can start training your algorithm to make
movie recommendations for yourself. 

In the cell below, you can enter your own movie choices. The algorithm will then make recommendations for you! We have filled out some values according to our preferences, but after you have things working with our choices, you should change this to match your tastes.


In [52]:
num_movies=ratings.shape[0]
my_ratings = np.zeros(num_movies)          #  Initialize my ratings

# Check the file small_movie_list.csv for id of each movie in our dataset
# For example, Toy Story 3 (2010) has ID 2700, so to rate it "5", you can set
my_ratings[8877] = 5 

#Or suppose you did not enjoy Persuasion (2007), you can set
my_ratings[6577] = 2;

# We have selected a few movies we liked / did not like and the ratings we
# gave are as follows:
my_ratings[5208:5209]  = 5   # Lord of the Rings: The Return of the King, The
my_ratings[7657:7658]  = 5   # Shrek (2001)
my_ratings[4326:4327] = 3   # Inception
my_ratings[4337] = 5   # Incredibles, The (2004)
my_ratings[400]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[3797]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
my_ratings[3790]  = 5   # Harry Potter and the Chamber of Secrets (2002)
my_ratings[2794]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[5242] = 1   # Louis Theroux: Law & Disorder (2008)
my_ratings[6197] = 1   # Nothing to Declare (Rien à déclarer)
my_ratings[6649]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)
my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

print('\nNew user ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0 :
        print(f'Rated {my_ratings[i]} for  {X_featt.loc[i,"title"]}');


New user ratings:

Rated 2.0 for  Amelie ,Fabuleux destin d'Amélie Poulain, Le) 
Rated 3.0 for  Eternal Sunshine of the Spotless Mind 
Rated 5.0 for  Harry Potter and the Chamber of Secrets 
Rated 5.0 for  Harry Potter and the Sorcerer's Stone ,a.k.a. Harry Potter and the Philosopher's Stone) 
Rated 3.0 for  Inception 
Rated 5.0 for  Incredibles, The 
Rated 5.0 for  Lord of the Rings: The Return of the King, The 
Rated 1.0 for  Louis Theroux: Law & Disorder 
Rated 1.0 for  Nothing to Declare ,Rien à déclarer) 
Rated 2.0 for  Persuasion 
Rated 5.0 for  Pirates of the Caribbean: The Curse of the Black Pearl 
Rated 5.0 for  Shrek 
Rated 5.0 for  Toy Story 3 


# Recommendations
Below, we compute the ratings for all the movies and users and display the movies that are recommended. These are based on the movies and ratings entered as `my_ratings[]` above. To predict the rating of movie $i$ for user $j$, you compute $\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)}$. This can be computed for all ratings using matrix multiplication.

In [53]:
# Make a prediction using trained weights and biases
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

#restore the mean
#(ratings.T-mean).T
pm = (p.T + mean).T
print(pm.shape)
my_predictions = pm[:,0]

# sort predictions
ix = tf.argsort(my_predictions, direction='DESCENDING')

for i in range(17):
    j = ix[i]
    if j not in my_rated:
        print(f'Predicting rating {my_predictions[j]:0.2f} for movie {X_featt.loc[int(j),"title"]}')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f'Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {X_featt.loc[int(i),"title"]}')

(9724, 610)
Predicting rating 7.33 for movie Green Card 
Predicting rating 7.19 for movie Kingdom of Heaven 
Predicting rating 7.15 for movie Humpday 
Predicting rating 7.04 for movie Big Red One, The 
Predicting rating 6.95 for movie Beat Street 
Predicting rating 6.77 for movie Killing Zoe 
Predicting rating 6.71 for movie Stealing Beauty 
Predicting rating 6.70 for movie Trading Places 
Predicting rating 6.63 for movie Best Man, The 
Predicting rating 6.63 for movie Good Night, and Good Luck. 
Predicting rating 6.62 for movie Adventures of Tintin, The 
Predicting rating 6.52 for movie Assassination of Richard Nixon, The 
Predicting rating 6.31 for movie Band of Brothers 
Predicting rating 6.26 for movie Camp X-Ray 
Predicting rating 6.23 for movie Amy's O ,a.k.a. Amy's Orgasm) 
Predicting rating 6.23 for movie Brothers 
Predicting rating 6.23 for movie Because of Winn-Dixie 


Original vs Predicted ratings:

Original 2.0, Predicted 2.97 for Amelie ,Fabuleux destin d'Amélie Poulain, 

In [54]:
filter=(X_featt["num_rating"] > 20)
X_featt["pred"] = my_predictions
X_featt = X_featt.reindex(columns=["pred", "mean_rating", "num_rating", "title",'genres','movieId'])
X_featt.loc[ix[:300]].loc[filter].sort_values("mean_rating", ascending=False)

,pred,mean_rating,num_rating,title,genres,movieId
9123,4.302190,4.237745,204,"Usual Suspects, The","[Crime, Mystery, Thriller]",50
8797,4.956634,4.120690,58,To Kill a Mockingbird,[Drama],1207
2943,5.113784,4.116022,181,Fargo,"[Comedy, Crime, Drama, Thriller]",608
5436,5.976701,4.106061,33,Manhattan,"[Comedy, Drama, Romance]",1244
405,4.925827,4.056373,204,American Beauty,"[Drama, Romance]",2858
9167,5.254558,4.025000,60,Vertigo,"[Drama, Mystery, Romance, Thriller]",903
5484,3.694313,3.887324,71,Mary Poppins,"[Children, Comedy, Fantasy, Musical]",1028
9124,4.885781,3.885000,100,V for Vendetta,"[Action, Sci-Fi, Thriller, IMAX]",44191
9542,4.882019,3.880435,92,"Wizard of Oz, The","[Adventure, Children, Fantasy, Musical]",919
2317,4.207749,3.833333,21,Delicatessen,"[Comedy, Drama, Romance]",1175


In [55]:
movies,links,ratings,tags = Load_Data()
X=Preprocessing(movies,links)
d=X
X_feat,Y_ = Preprocessing2(X,ratings,tags,d)


X_feat.drop('genress',axis=1,inplace=True)


X_featt=X_feat.copy(deep=True)
X_featt.sort_values(by='movieId',inplace=True)
X_feat.set_index('movieId',inplace=True,drop=True)
X_feat.sort_index(inplace=True)
X_feat.drop('title',axis=1,inplace=True)


Y_.drop(['genres','imdbId','tmdbId','title','year','timestamp'],axis=1,inplace=True)
ratings=Y_.groupby(['movieId', 'userId'])['rating'].first().unstack()
ratings.sort_index(inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


# Let's prepare to train the model. Initialize the parameters X and W and B randomly and select the Adam optimizer.

In [56]:
num_features = 21

In [57]:

R=ratings.isnull().values
R=~np.array(R)
R = np.multiply(R, 1)


    
    
mean = np.nanmean(ratings, axis=1)
Ynorm = (ratings.T-mean).T    
W=np.random.randn(Ynorm.shape[1]   ,X.shape[1]   )
b=np.random.randn(1,Ynorm.shape[1]    )

Ynorm=pd.DataFrame(Ynorm)
Ynorm=Ynorm.fillna(0)


Ynorm=np.array(Ynorm)

num_movies, num_users = Ynorm.shape


tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')
print(X.shape)

J = cofi_cost_func_v(X, W, b, Ynorm, R, 1.5);
print(f"Cost: {J:0.2f}")
J = cofi_cost_func_v(X, W, b, Ynorm, R, 0);
print(f"Cost: {J:0.2f}")


(9724, 21)
Cost: 1326108.52
Cost: 1163129.72


# Here we Use PiecewiseConstantDecay To Reduce learning Rate with a specific schedule

In [58]:
iterations = 5000
lambda_ = 1

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=9e-1,decay_steps=1000,decay_rate=0.12)

optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )
    
    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )
    
    # Log periodically.
    if iter % 100 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 1271782.3
Training loss at iteration 100: 9949.6
Training loss at iteration 200: 9432.8
Training loss at iteration 300: 9304.7
Training loss at iteration 400: 9240.0
Training loss at iteration 500: 9205.8
Training loss at iteration 600: 9184.7
Training loss at iteration 700: 9168.7
Training loss at iteration 800: 9156.2
Training loss at iteration 900: 9147.0
Training loss at iteration 1000: 9140.1
Training loss at iteration 1100: 9134.8
Training loss at iteration 1200: 9130.8
Training loss at iteration 1300: 9127.7
Training loss at iteration 1400: 9125.4
Training loss at iteration 1500: 9123.5
Training loss at iteration 1600: 9122.0
Training loss at iteration 1700: 9120.8
Training loss at iteration 1800: 9119.8
Training loss at iteration 1900: 9119.0
Training loss at iteration 2000: 9118.3
Training loss at iteration 2100: 9117.7
Training loss at iteration 2200: 9117.2
Training loss at iteration 2300: 9116.8
Training loss at iteration 2400: 9116.4
Training 

In [59]:
num_movies=ratings.shape[0]
my_ratings = np.zeros(num_movies)          #  Initialize my ratings

# Check the file small_movie_list.csv for id of each movie in our dataset
# For example, Toy Story 3 (2010) has ID 2700, so to rate it "5", you can set
my_ratings[8877] = 5 

#Or suppose you did not enjoy Persuasion (2007), you can set
my_ratings[6577] = 2;

# We have selected a few movies we liked / did not like and the ratings we
# gave are as follows:
my_ratings[5208:5209]  = 5   # Lord of the Rings: The Return of the King, The
my_ratings[7657:7658]  = 5   # Shrek (2001)
my_ratings[4326:4327] = 3   # Inception
my_ratings[4337] = 5   # Incredibles, The (2004)
my_ratings[400]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[3797]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
my_ratings[3790]  = 5   # Harry Potter and the Chamber of Secrets (2002)
my_ratings[2794]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[5242] = 1   # Louis Theroux: Law & Disorder (2008)
my_ratings[6197] = 1   # Nothing to Declare (Rien à déclarer)
my_ratings[6649]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)
my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

print('\nNew user ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0 :
        print(f'Rated {my_ratings[i]} for  {X_featt.loc[i,"title"]}');


New user ratings:

Rated 2.0 for  Amelie ,Fabuleux destin d'Amélie Poulain, Le) 
Rated 3.0 for  Eternal Sunshine of the Spotless Mind 
Rated 5.0 for  Harry Potter and the Chamber of Secrets 
Rated 5.0 for  Harry Potter and the Sorcerer's Stone ,a.k.a. Harry Potter and the Philosopher's Stone) 
Rated 3.0 for  Inception 
Rated 5.0 for  Incredibles, The 
Rated 5.0 for  Lord of the Rings: The Return of the King, The 
Rated 1.0 for  Louis Theroux: Law & Disorder 
Rated 1.0 for  Nothing to Declare ,Rien à déclarer) 
Rated 2.0 for  Persuasion 
Rated 5.0 for  Pirates of the Caribbean: The Curse of the Black Pearl 
Rated 5.0 for  Shrek 
Rated 5.0 for  Toy Story 3 


In [60]:
# Make a prediction using trained weights and biases
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

#restore the mean
#(ratings.T-mean).T
pm = (p.T + mean).T
print(pm.shape)
my_predictions = pm[:,0]

# sort predictions
ix = tf.argsort(my_predictions, direction='DESCENDING')

for i in range(17):
    j = ix[i]
    if j not in my_rated:
        print(f'Predicting rating {my_predictions[j]:0.2f} for movie {X_featt.loc[int(j),"title"]}')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f'Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {X_featt.loc[int(i),"title"]}')

(9724, 610)
Predicting rating 6.35 for movie Under the Sand 
Predicting rating 6.28 for movie Things to Do in Denver When You're Dead 
Predicting rating 6.20 for movie 9to5: Days in Porn ,a.k.a. 9 to 5: Days in Porn) 
Predicting rating 6.19 for movie Phantasm III: Lord of the Dead 
Predicting rating 6.19 for movie Larry David: Curb Your Enthusiasm 
Predicting rating 6.17 for movie World Is Not Enough, The 
Predicting rating 6.10 for movie Trading Places 
Predicting rating 6.08 for movie Aces: Iron Eagle III 
Predicting rating 6.07 for movie Before the Devil Knows You're Dead 
Predicting rating 6.05 for movie She's Out of Control 
Predicting rating 6.05 for movie Stepford Wives, The 
Predicting rating 6.05 for movie The Unauthorized Saved by the Bell Story 
Predicting rating 6.03 for movie The Darkest Minds 
Predicting rating 5.99 for movie Rum Diary, The 
Predicting rating 5.99 for movie Dr. T and the Women 
Predicting rating 5.98 for movie Head Above Water 
Predicting rating 5.97 for 

In [61]:
# filter=(X_featt["num_rating"] > 20)
X_featt["pred"] = my_predictions
X_featt = X_featt.reindex(columns=["pred", "mean_rating", "num_rating", "title",'genres','movieId'])
X_featt.loc[ix[:300]].loc[filter].sort_values("mean_rating", ascending=False)

,pred,mean_rating,num_rating,title,genres,movieId
9123,4.452754,4.237745,204,"Usual Suspects, The","[Crime, Mystery, Thriller]",50
8797,5.239622,4.120690,58,To Kill a Mockingbird,[Drama],1207
2943,5.146356,4.116022,181,Fargo,"[Comedy, Crime, Drama, Thriller]",608
5436,3.927586,4.106061,33,Manhattan,"[Comedy, Drama, Romance]",1244
9167,5.742305,4.025000,60,Vertigo,"[Drama, Mystery, Romance, Thriller]",903
7474,4.227455,3.976190,21,"Secret Life of Walter Mitty, The","[Comedy, Romance, Thriller]",7826
5484,2.899633,3.887324,71,Mary Poppins,"[Children, Comedy, Fantasy, Musical]",1028
9124,4.978462,3.885000,100,V for Vendetta,"[Action, Sci-Fi, Thriller, IMAX]",44191
9542,5.439939,3.880435,92,"Wizard of Oz, The","[Adventure, Children, Fantasy, Musical]",919
1227,4.819963,3.809524,84,"Blues Brothers, The","[Action, Comedy, Musical]",1220


# 

# 

# Deep Learning for Content-Based Filtering


# <a name="3"></a>
## Content-based filtering with a neural network

In the collaborative filtering lab, you generated two vectors, a user vector and an item/movie vector whose dot product would predict a rating. The vectors were derived solely from the ratings.   

Content-based filtering also generates a user and movie feature vector but recognizes there may be other information available about the user and/or movie that may improve the prediction.

<a name="3.1"></a>
###  Training Data
The movie content provided to the network is a combination of the original data and some 'engineered features'. Recall the feature engineering discussion and lab from Course 1, Week 2, lab 4. The original features are the year the movie was released and the movie's genre's presented as a one-hot vector. There are 14 genres. The engineered feature is an average rating derived from the user ratings. 

The user content is composed of engineered features. A per genre average rating is computed per user. Additionally, a user id, rating count and rating average are available but not included in the training or prediction content. They are carried with the data set because they are useful in interpreting data.

The training set consists of all the ratings made by the users in the data set. Some ratings are repeated to boost the number of training examples of underrepresented genre's. The training set is split into two arrays with the same number of entries, a user array and a movie/item array.  

Below, let's load and display some of the data.

# Data preprocessing_Filtering1
Merge movies and links Data to Get a one DataFrame to can doing operation on it
Split title to Movie_title and Movie_Year to Can doing Operation and Recommendation Base on alot of Columns including Year of Movie¶

# Split genres Column To get column of List of Movie genres like Adventure|Animation|Children|Comedy|Fantasy => ['Adventure','Animation','Children','Comedy','Fantasy']

### Also Split X['title'] to X['title'] and X['Year'] to make a recommendation more accurate about year of movie had been made
### Doing some Groups to Get a spesific columns and row which it has values df_item_train & df_user_train to Make a best and a correct Deep Learning for Content-Based Filtering to make the best recommendation 

In [62]:
def preprocessing_Filtering1(movies,links,ratings,tags):
    X=movies['movieId'],movies['genres'],links['imdbId'],links['tmdbId'],movies['title']
    X=pd.DataFrame(X).T
    X['imdbId']=X['imdbId'].astype(float)
    X['tmdbId']=X['tmdbId'].astype(float)
    #X=movies
    tit=[]
    year=[]
    genres=[]
    for i in X.index:
        genres.append(np.array(X['genres'][i].split('|')))
        if str(((X['title'][i].split('(')[-1].replace(')','')).replace(" ", "")).split('–')[-1]).isnumeric()== True:    
            tit.append(','.join(X['title'][i].split('(')[:-1]))
            year.append(((X['title'][i].split('(')[-1].replace(')','')).replace(" ", "")).split('–')[-1])
        else:
            tit.append(','.join(X['title'][i].split('(')[:]))
            year.append(0)
    tit=np.array(tit)    
    year=np.array(year)    
    X=X.drop(['title'],axis=1)
    X['title']=tit
    X['year']=year

    X['year']=X['year'].astype(int)
    X['genress']=genres
    
    X=pd.merge(X,ratings,on='movieId')

    n=X[['userId','rating']].groupby('userId',as_index=False).count()
    n.columns=['userId','rating_count']
    X=X.merge(n,on='userId',how='left')


    X['movieId']=X['movieId'].astype(int)

    d2= X[['userId','rating']].groupby(['userId'],as_index=False).mean()
    d2.columns=['userId','user_mean_rating']

    X=X.merge(d2,on='userId',how='left')

    d2= X[['movieId','rating']].groupby(['movieId'],as_index=False).mean()
    d2.columns=['movieId','movie_mean_rating']

    X=X.merge(d2,on='movieId',how='left')


    return X

In [63]:

movies , links,ratings,tags = Load_Data()
X=preprocessing_Filtering1(movies,links,ratings,tags)

df_item_train= X[['movieId','genress','year','movie_mean_rating']]

mlb = MultiLabelBinarizer(sparse_output=True)

df_item_train = df_item_train.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df_item_train.pop('genress')),
                index=df_item_train.index,
                columns=mlb.classes_))



m=X[['genres','rating','userId']]
col=df_item_train.columns
col=list(col)
col[:3]=['userId','rating_count','user_mean_rating']
df_user_train=pd.DataFrame(df_item_train.values,columns=col)


df_user_train['rating_count']=X['rating_count']
df_user_train['userId']=X['userId']
df_user_train['user_mean_rating']=X['user_mean_rating']

df_user_train.columns.values[:3]=['userId','rating_count','user_mean_rating']
for i in df_user_train.columns[3:]:
    df_user_train[i]=m[m['genres'].str.contains(i)]['rating']

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [64]:
df_user_train

,userId,rating_count,user_mean_rating,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,232,4.366379,NaN,NaN,4.0,4.0,4.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,44,3.636364,NaN,NaN,4.0,4.0,4.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,152,3.230263,NaN,NaN,4.5,4.5,4.5,4.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15,135,3.448148,NaN,NaN,2.5,2.5,2.5,2.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17,105,4.209524,NaN,NaN,4.5,4.5,4.5,4.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,184,134,3.705224,NaN,4.0,NaN,4.0,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100832,184,134,3.705224,NaN,NaN,NaN,3.5,NaN,3.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100833,184,134,3.705224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100834,184,134,3.705224,NaN,3.5,NaN,3.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
df_item_train

,movieId,year,movie_mean_rating,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,3.92093,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1995,3.92093,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1995,3.92093,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1995,3.92093,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1995,3.92093,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,193581,2017,4.00000,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
100832,193583,2017,3.50000,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
100833,193585,2017,3.50000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100834,193587,2018,3.50000,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Fill null values with zero rate to prevent affect on original Data 

In [66]:
df_user_train.fillna(0,inplace=True)
user_train=np.array(df_user_train)

In [67]:
#df_item_train.fillna(0,inplace=True)
item_train=np.array(df_item_train)

In [68]:
y_train=np.array(X['rating'])
y_train

array([4. , 4. , 4.5, ..., 3.5, 3.5, 4. ])

In [69]:
y_train.shape

(100836,)

In [70]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


# Preparing the training data
### We'll scale the input features using the [scikit learn StandardScaler]

In [71]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [72]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)

In [73]:
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (80668, 23)
movie/item test data shape: (20168, 23)


In [74]:
num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items

# 

# Neural Network for content-based filtering
Now, let's construct a neural network as described in the figure above. It will have two networks that are combined by a dot product. You will construct the two networks. In this example, they will be identical. Note that these networks do not need to be the same. If the user content was substantially larger than the movie content, you might elect to increase the complexity of the user network relative to the movie network. In this case, the content is similar, so the networks are the same.

In [75]:
# GRADED_CELL
# UNQ_C1

num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  
  tf.keras.layers.Dense(256,activation='relu'),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(32)
  
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256,activation='relu'),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(32)
  
  
    ### END CODE HERE ###  
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 22)]         0           []                               
                                                                                                  
 sequential (Sequential)        (None, 32)           42400       ['input_1[0][0]']                
                                                                                                  
 sequential_1 (Sequential)      (None, 32)           42912       ['input_2[0][0]']                
                                                                                              

#### We will use a mean squared error loss and an Adam optimizer.

In [76]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [77]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=10)

Epoch 1/10
2521/2521 [==============================] - 3s 1ms/step - loss: 0.0097
Epoch 2/10
2521/2521 [==============================] - 3s 1ms/step - loss: 0.0025
Epoch 3/10
2521/2521 [==============================] - 3s 1ms/step - loss: 0.0017
Epoch 4/10
2521/2521 [==============================] - 3s 1ms/step - loss: 0.0013
Epoch 5/10
2521/2521 [==============================] - 3s 1ms/step - loss: 0.0010
Epoch 6/10
2521/2521 [==============================] - 3s 1ms/step - loss: 8.5464e-04
Epoch 7/10
2521/2521 [==============================] - 3s 1ms/step - loss: 7.1163e-04
Epoch 8/10
2521/2521 [==============================] - 3s 1ms/step - loss: 6.6952e-04
Epoch 9/10
2521/2521 [==============================] - 3s 1ms/step - loss: 5.3649e-04
Epoch 10/10
2521/2521 [==============================] - 3s 1ms/step - loss: 4.8427e-04


In [78]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

631/631 [==============================] - 1s 765us/step - loss: 4.7376e-04


0.0004737605631817132

In [79]:
df_user_train.columns

Index(['userId', 'rating_count', 'user_mean_rating', '(no genres listed)',
       'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')

# Predictions
Below, you'll use your model to make predictions in a number of circumstances.

## Predictions for a new user
First, we'll create a new user and have the model suggest movies for that user. After you have tried this on the example user content, feel free to change the user content to match your own preferences and see what the model suggests. 

In [80]:
new_user_id = 5000
new_rating_count = 4
new_rating_ave = 0.0
no_genres_listed=0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 5.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_Film_Noir = 0.0
new_horror = 0.0
new_IMAX = 0.0
new_Musical = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_War=5.0
new_Western = 0.0

user_vecc = np.array([[new_user_id, new_rating_count, new_rating_ave,no_genres_listed,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_Film_Noir,new_horror, new_IMAX,new_Musical,new_mystery,
                      new_romance, new_scifi, new_thriller,new_War,new_Western]])
user_vecc.shape

(1, 23)

In [81]:
df_item_train.sort_values(by='movieId')
item_vecs=df_item_train.groupby('movieId',as_index=False).first()[0+7000:7000+847]


In [82]:
item_vecs 

,movieId,year,movie_mean_rating,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
7000,68347,2009,4.000000,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
7001,68358,2009,3.864407,0,1,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
7002,68444,2008,3.500000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7003,68480,1980,1.500000,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
7004,68486,2009,4.000000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7842,93855,2011,4.500000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7843,93980,2012,1.250000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7844,93982,2012,2.000000,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
7845,93988,2004,4.500000,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


### The new user enjoys movies from the adventure, fantasy genres ...etc  Let's find the top-rated movies for the new user.
### Below, we'll use a set of movie/item vectors, `item_vecs` that have a vector for each movie in the training/test set. This is matched with the new user vector above and the scaled vectors are used to predict ratings for all the movies.

In [83]:
# generate and replicate the user vector to match the number movies in the data set.
#user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale our user and item vectors
item_vecss=np.array(item_vecs)
#mean = np.nanmean(df_user_train, axis=0)
#user_vecc=user_vec+mean
user_vec=np.array(list(user_vecc)*len(item_vecs))
#user_vec=user_vec.reshape(len(item_vecs),23)

suser_vecs = scalerUser.transform(user_vec)
sitem_vecs = scalerItem.transform(item_vecss)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs.iloc[sorted_index]  

27/27 [==============================] - 0s 1ms/step


In [84]:
s=sorted_items[['movieId','year','movie_mean_rating','Action']]
s['Action']=sorted_ypu
s.columns=['movieId','year','movie_mean_rating','y_p']
m=X[X['movieId'].isin(s['movieId'])].groupby('movieId',as_index=False).first()
f=s.merge(m[['movieId','title','genres']],on='movieId',how='outer')
f.sort_values(by='y_p',inplace=True,ascending=False)
f

<ipython-input-84-942bc2ec7e35>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s['Action']=sorted_ypu


,movieId,year,movie_mean_rating,y_p,title,genres
0,71264,2009,3.529412,4.998248,Cloudy with a Chance of Meatballs,Animation|Children|Fantasy|IMAX
1,91483,2012,1.500000,4.997392,Bullet to the Head,Action|Crime|Film-Noir
2,80615,2010,4.000000,4.996778,Legend of the Guardians: The Owls of Ga'Hoole,Adventure|Animation|Fantasy|IMAX
3,86864,1961,3.000000,4.996505,"Mothra ,Mosura)",Adventure|Fantasy|Sci-Fi
4,75947,1962,2.000000,4.996068,Mondo Cane,Documentary
...,...,...,...,...,...,...
842,74791,2009,3.750000,4.271183,"Town Called Panic, A ,Panique au village)",Animation
843,70687,2009,2.750000,4.254714,Paper Heart,Comedy|Documentary|Drama|Romance
844,82857,2009,4.500000,4.122827,Sweetgrass,Adventure|Documentary|Western
845,69469,2009,5.000000,3.859278,Garfield's Pet Force,Animation


# Predictions for an existing user.
Let's look at the predictions for "user 599", one of the users in the data set. We can compare the predicted ratings with the model's ratings.

In [85]:
userr=599
user_vecs = df_user_train[df_user_train['userId']==599]
y_vecs =y_train_unscaled[user_vecs.index]

In [86]:
s=np.array([user_vecs]*int(len(item_vecs)/len(user_vecs)+1))
s=s.reshape(-1,23)
user_vecs=s[:len(item_vecs)]
user_vecs=pd.DataFrame(user_vecs)
user_vecs.fillna(0,inplace=True)
user_vecs=np.array(user_vecs)
s=np.array(list(y_vecs)*int(len(item_vecs)/len(y_vecs)+1))
y_vecs=s[:len(item_vecs)]
item_vecss=np.array(item_vecs)

suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecss)
print(len(suser_vecs))
len(sitem_vecs)
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# make a prediction

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs.iloc[sorted_index]  #using unscaled vectors for display
sorted_user  = user_vecs[sorted_index]
sorted_y     = y_vecs[sorted_index]


847
27/27 [==============================] - 0s 854us/step


In [87]:
s=sorted_items[['movieId','year','movie_mean_rating','Action']]
s['Action']=sorted_ypu
s

<ipython-input-87-ef86e29ed1a6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s['Action']=sorted_ypu


,movieId,year,movie_mean_rating,Action
7406,80615,2010,4.000000,4.997943
7340,78574,2010,3.437500,4.996367
7794,92494,2004,5.000000,4.992432
7645,88746,2011,3.500000,4.991342
7107,71135,2009,3.400000,4.984822
...,...,...,...,...
7527,85025,2011,2.000000,0.516210
7694,90345,2011,3.500000,0.508774
7615,87529,2011,3.111111,0.504583
7771,91947,2009,4.500000,0.502253


In [88]:
s=sorted_items[['movieId','year','movie_mean_rating','Action']]
s['Action']=sorted_ypu
s.columns=['movieId','year','movie_mean_rating','y_p']
m=X[X['movieId'].isin(s['movieId'])].groupby('movieId',as_index=False).first()
f=s.merge(m[['movieId','title','genres','userId']],on='movieId',how='outer')
f.sort_values(by='y_p',inplace=True,ascending=False)
f[f['userId']==599][:50]

<ipython-input-88-0ffb185ff3e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s['Action']=sorted_ypu


,movieId,year,movie_mean_rating,y_p,title,genres,userId
11,91981,2010,2.50,4.933990,"Sacrifice ,Zhao shi gu er)",Drama,599
30,71484,2009,2.50,4.687236,Metropia,Animation|Sci-Fi,599
38,81791,2010,3.00,4.567699,Somewhere,Drama,599
39,93208,1990,2.00,4.564452,Mickey's The Prince and the Pauper,Animation|Children,599
40,70703,1992,2.00,4.548485,Christopher Columbus: The Discovery,Adventure,599
102,71327,2009,3.50,3.980034,Bright Star,Drama|Romance,599
125,89085,2011,2.50,3.843346,"Debt, The",Drama|Thriller,599
128,68650,2009,2.00,3.827513,Powder Blue,Drama,599
171,83086,2010,2.50,3.565780,Burlesque,Drama|Musical|Romance,599
172,74696,2009,3.00,3.560960,Last Train Home,Documentary|Drama,599


In [89]:
f.groupby('userId').count()

,movieId,year,movie_mean_rating,y_p,title,genres
userId,,,,,,
2,10,10,10,10,10,10
3,2,2,2,2,2,2
10,30,30,30,30,30,30
15,9,9,9,9,9,9
16,1,1,1,1,1,1
...,...,...,...,...,...,...
596,4,4,4,4,4,4
599,47,47,47,47,47,47
605,3,3,3,3,3,3


# Finding Similar Items
The neural network above produces two feature vectors, a user feature vector $v_u$, and a movie feature vector, $v_m$. These are 32 entry vectors whose values are difficult to interpret. However, similar items will have similar vectors. This information can be used to make recommendations. For example, if a user has rated "Toy Story 3" highly, one could recommend similar movies by selecting movies with similar movie feature vectors.

A similarity measure is the squared distance between the two vectors $ \mathbf{v_m^{(k)}}$ and $\mathbf{v_m^{(i)}}$ :
$$\left\Vert \mathbf{v_m^{(k)}} - \mathbf{v_m^{(i)}}  \right\Vert^2 = \sum_{l=1}^{n}(v_{m_l}^{(k)} - v_{m_l}^{(i)})^2\tag{1}$$

In [90]:
# GRADED_FUNCTION: sq_dist
# UNQ_C2
def sq_dist(a,b):
    """
    Returns the squared distance between two vectors
    Args:
      a (ndarray (n,)): vector with n features
      b (ndarray (n,)): vector with n features
    Returns:
      d (float) : distance
    """
    ### START CODE HERE ###     
    d=np.sum((a-b)**2)
    ### END CODE HERE ###     
    return d

##### A matrix of distances between movies can be computed once when the model is trained and then reused for new recommendations without retraining. The first step, once a model is trained, is to obtain the movie feature vector,  𝑣𝑚 , for each of the movies. To do this, we will use the trained item_NN and build a small model to allow us to run the movie vectors through it to generate  𝑣𝑚 .

In [91]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))    # input layer
vm_m = item_NN(input_item_m)                                       # use the trained item_NN
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                        # incorporate normalization as was done in the original model
model_m = tf.keras.Model(input_item_m, vm_m)                                
model_m.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 22)]              0         
                                                                 
 sequential_1 (Sequential)   (None, 32)                42912     
                                                                 
 tf.math.l2_normalize_2 (TFO  (None, 32)               0         
 pLambda)                                                        
                                                                 
Total params: 42,912
Trainable params: 42,912
Non-trainable params: 0
_________________________________________________________________


 Once you have a movie model, you can create a set of movie feature vectors by using the model to predict using a set of item/movie vectors as input. `item_vecs` is a set of all of the movie vectors. It must be scaled to use with the trained model. The result o

In [92]:
# item_vecss=np.array(item_vecs)

scaled_item_vecs = scalerItem.transform(item_vecss)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

27/27 [==============================] - 0s 1ms/step
size of all predicted movie feature vectors: (847, 32)


We can then find the closest movie by finding the minimum along each row. We will make use of [numpy masked arrays](https://numpy.org/doc/1.21/user/tutorial-ma.html) to avoid selecting the same movie. The masked values along the diagonal won't be included in the computation.

In [93]:
count = 50  # number of movies to display
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])
        
m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal

disp = [["movie1", "genres", "movie2", "genres"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    movie1_id = int(item_vecs.iloc[i,0])
    movie2_id = int(item_vecs.iloc[min_idx,0])
    disp.append( [str(str(set(X[X['movieId']==movie1_id]['title'])).strip("{},''")+' '+str(set(X[X['movieId']==movie1_id]['year'])).strip("{},''")).strip("''"),str(set(X[X['movieId']==movie1_id]['genres'])).strip("{},''"),
                 str(str(set( X[X['movieId']==movie2_id]['title'])).strip("{},''")+' '+str(set(X[X['movieId']==movie1_id]['year'])).strip("{},''")).strip("''"), str(set(X[X['movieId']==movie1_id]['genres'])).strip("{},''")]
               )
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
table

movie1,genres,movie2,genres
Sin Nombre 2009,Crime|Drama|Thriller,Harry Brown 2009,Crime|Drama|Thriller
Star Trek 2009,Action|Adventure|Sci-Fi|IMAX,"Avengers, The 2009",Action|Adventure|Sci-Fi|IMAX
"Great Buck Howard, The 2008",Comedy,"""Slammin' Salmon, The "" 2008",Comedy
"""Flatfoot on the Nile ,Piedone d'Egitto) "" 1980",Action|Comedy|Crime,One for the Money 1980,Action|Comedy|Crime
"Red Cliff Part II ,Chi Bi Xia: Jue Zhan Tian Xia) 2009",Action|Drama|War,"New One-Armed Swordsman, The ,Xin du bi dao) 2009",Action|Drama|War
Earth 2007,Documentary,"9to5: Days in Porn ,a.k.a. 9 to 5: Days in Porn) 2007",Documentary
Stanley Kubrick: A Life in Pictures 2001,Documentary,Earthlings 2001,Documentary
Crossing Over 2009,Drama,Powder Blue 2009,Drama
Angels & Demons 2009,Crime|Drama|Mystery|Thriller,G.I. Joe: The Rise of Cobra 2009,Crime|Drama|Mystery|Thriller
Scorpio 1973,Action|Drama|Thriller,Prison Break: The Final Break 1973,Action|Drama|Thriller


# 

# Please don't forget to tell us your opinion about this project

# Congratulations! 


# 